project- Predict No Shows for Medical  Appointments

In [ ]:
# import important libraries
import pandas as pd
import numpy as np
#https://www.youtube.com/watch?v=BFgbfk3LYtw

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_health= pd.read_csv("/content/drive/MyDrive/Datasets/ML p2/train_health.csv")
test_health = pd.read_csv("/content/drive/MyDrive/Datasets/ML p2/test_health.csv")
medical_history = pd.read_csv("/content/drive/MyDrive/Datasets/ML p2/medical_history (1).csv")
demographic_details = pd.read_csv("/content/drive/MyDrive/Datasets/ML p2/demographic_details.csv")

* check  target column No-show in train_health
* underatand all features and column

2. Data Preprocessing
*Drop dublicates of train and test
*merge other data with train and test
*Handle Missing Values
*Feature Engineering- (levelEncoder, OneHotEncoder)
*Feature Scaling

In [ ]:
# handling missing value
test_data= test_health.dropna()

In [ ]:
test_data.shape

(10854, 5)

In [ ]:
# Check entire DataFrame
print(train_health.isnull().values.any())

False


In [ ]:
print(train_health.duplicated().sum())


0


In [ ]:
print(test_health.duplicated().sum())

10853


In [ ]:
print(test_data.isnull().values.any())

False


In [ ]:
train_health.head()


,PatientId,AppointmentID,ScheduledDay,AppointmentDay,SMS_received,No-show
0,2.987250e+13,5642903,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,0,No
1,5.589980e+14,5642503,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,0,No
2,8.679510e+11,5642828,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,0,No
3,8.841190e+12,5642494,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,0,No
4,9.598510e+13,5626772,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,0,No


In [ ]:
print(medical_history.isnull().values.any())

False


In [ ]:
medical_history.head()

,PatientId,Hipertension,Diabetes,Alcoholism,Handcap
0,2.987250e+13,1,0,0,0
1,5.589980e+14,0,0,0,0
2,4.262960e+12,0,0,0,0
3,8.679510e+11,0,0,0,0
4,8.841190e+12,1,1,0,0


In [ ]:
print(demographic_details.isnull().values.any())
# fillina but no need because  isnull is 0
# demo = demographic_details.fillna('0', inplace=False)

False


In [ ]:
demographic_details.head()

,PatientId,Gender,Age,Neighbourhood,Scholarship
0,2.990000e+13,F,62,JARDIM DA PENHA,0
1,5.590000e+14,M,56,JARDIM DA PENHA,0
2,4.260000e+12,F,62,MATA DA PRAIA,0
3,8.680000e+11,F,8,PONTAL DE CAMBURI,0
4,8.840000e+12,F,56,JARDIM DA PENHA,0


In [ ]:
#Scholarship distribution: binary (0 - No, 1 - Yes)
print(demographic_details['Scholarship'].value_counts())


Scholarship
0    56511
1     5788
Name: count, dtype: int64


In [ ]:
#Age distribution:
print(demographic_details['Age'].describe())


count    62299.000000
mean        36.704891
std         23.531372
min         -1.000000
25%         17.000000
50%         36.000000
75%         56.000000
max        115.000000
Name: Age, dtype: float64


In [ ]:
demographic_details['Age'].isnull().sum()

0

In [ ]:
# gender .Map() or .replace()
demographic_details['Gender'] = demographic_details['Gender'].map({'M': 1, 'F': 0})

In [ ]:
demographic_details.head()

,PatientId,Gender,Age,Neighbourhood,Scholarship
0,2.990000e+13,0,62,JARDIM DA PENHA,0
1,5.590000e+14,1,56,JARDIM DA PENHA,0
2,4.260000e+12,0,62,MATA DA PRAIA,0
3,8.680000e+11,0,8,PONTAL DE CAMBURI,0
4,8.840000e+12,0,56,JARDIM DA PENHA,0


In [ ]:
# demographic_details['Neighbourhood'].unique()

In [ ]:
#Neighbourhood distribution: Categorical (different regions)

print(demographic_details['Neighbourhood'].value_counts())


Neighbourhood
JARDIM CAMBURI                 4192
MARIA ORTIZ                    3336
JARDIM DA PENHA                2406
RESISTÊNCIA                    2373
ITARARÉ                        2136
                               ... 
ILHA DO BOI                      22
AEROPORTO                         7
ILHA DO FRADE                     5
ILHAS OCEÂNICAS DE TRINDADE       2
PARQUE INDUSTRIAL                 1
Name: count, Length: 81, dtype: int64


In [ ]:
k = demographic_details['Neighbourhood'].value_counts()
cats=k[k>2000].index
# you can try slightly loweer cutoff to include dummy vars for more neighbourhoods
# check if that improves your model

print(f"Number of neighborhoods with more than 2000 occurrences: {len(cats)}")

Number of neighborhoods with more than 2000 occurrences: 5


In [ ]:
for cat in cats:
    name='Neighbourhood_'+cat
    demographic_details[name]=(demographic_details['Neighbourhood']==cat).astype(int)
del demographic_details['Neighbourhood']

In [ ]:
demographic_details.head()


,PatientId,Gender,Age,Scholarship,Neighbourhood_JARDIM CAMBURI,Neighbourhood_MARIA ORTIZ,Neighbourhood_JARDIM DA PENHA,Neighbourhood_RESISTÊNCIA,Neighbourhood_ITARARÉ
0,2.990000e+13,0,62,0,0,0,1,0,0
1,5.590000e+14,1,56,0,0,0,1,0,0
2,4.260000e+12,0,62,0,0,0,0,0,0
3,8.680000e+11,0,8,0,0,0,0,0,0
4,8.840000e+12,0,56,0,0,0,1,0,0


In [ ]:
demographic_details.shape

(62299, 9)

In [ ]:
#check distribution of NO-show:  binary (0 - No, 1 - Yes)
print(train_health['No-show'].value_counts())


No-show
No     79360
Yes    20130
Name: count, dtype: int64


merge

In [ ]:
comb=pd.merge(medical_history,demographic_details,how='outer',on='PatientId').drop_duplicates(subset='PatientId', keep='first')
comb.shape


(64929, 13)

In [ ]:
comb

,PatientId,Hipertension,Diabetes,Alcoholism,Handcap,Gender,Age,Scholarship,Neighbourhood_JARDIM CAMBURI,Neighbourhood_MARIA ORTIZ,Neighbourhood_JARDIM DA PENHA,Neighbourhood_RESISTÊNCIA,Neighbourhood_ITARARÉ
0,2.987250e+13,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.589980e+14,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.262960e+12,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.679510e+11,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8.841190e+12,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121678,9.610000e+11,NaN,NaN,NaN,NaN,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0
121679,5.410000e+14,NaN,NaN,NaN,NaN,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0
121680,3.710000e+11,NaN,NaN,NaN,NaN,0.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0
121681,6.610000e+11,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# to display df in chunk
'''comb_size = 100
for i in range(0, len(comb), comb_size):
    print(comb.iloc[i:i+comb_size])
'''

'comb_size = 100\nfor i in range(0, len(comb), comb_size):\n    print(comb.iloc[i:i+comb_size])\n'

In [ ]:
train_full=pd.merge(train_health, comb,how='left',on='PatientId')
test_full=pd.merge(test_data, comb,how='left',on='PatientId')
test_full.shape

(10854, 17)

In [ ]:
train_full.head()

,PatientId,AppointmentID,ScheduledDay,AppointmentDay,SMS_received,No-show,Hipertension,Diabetes,Alcoholism,Handcap,Gender,Age,Scholarship,Neighbourhood_JARDIM CAMBURI,Neighbourhood_MARIA ORTIZ,Neighbourhood_JARDIM DA PENHA,Neighbourhood_RESISTÊNCIA,Neighbourhood_ITARARÉ
0,2.987250e+13,5642903,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,0,No,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.589980e+14,5642503,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,0,No,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8.679510e+11,5642828,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,0,No,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.841190e+12,5642494,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,0,No,1.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.598510e+13,5626772,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,0,No,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# train.fillna(0, inplace=True)

#train.interpolate(method='linear', limit_direction='forward', inplace=True)
train_full.fillna(method='ffill', axis=0, inplace=True)  # Forward fill
train_full.fillna(method='bfill', axis=0, inplace=True)  # Backward fill

<ipython-input-117-05083ac6f6b3>:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_full.fillna(method='ffill', axis=0, inplace=True)  # Forward fill
<ipython-input-117-05083ac6f6b3>:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_full.fillna(method='bfill', axis=0, inplace=True)  # Backward fill


In [ ]:
train_full.head()

,PatientId,AppointmentID,ScheduledDay,AppointmentDay,SMS_received,No-show,Hipertension,Diabetes,Alcoholism,Handcap,Gender,Age,Scholarship,Neighbourhood_JARDIM CAMBURI,Neighbourhood_MARIA ORTIZ,Neighbourhood_JARDIM DA PENHA,Neighbourhood_RESISTÊNCIA,Neighbourhood_ITARARÉ
0,2.987250e+13,5642903,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,0,No,1.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.589980e+14,5642503,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,0,No,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8.679510e+11,5642828,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,0,No,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8.841190e+12,5642494,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,0,No,1.0,1.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9.598510e+13,5626772,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,0,No,1.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#test.fillna(0, inplace=True)   to replace NaN

#test .interpolate(method='linear', limit_direction='forward', inplace=True)

test_full.fillna(method='ffill', axis=0, inplace=True)  # Forward fill
test_full.fillna(method='bfill', axis=0, inplace=True)  # Backward fill


<ipython-input-119-a500bc606676>:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_full.fillna(method='ffill', axis=0, inplace=True)  # Forward fill
<ipython-input-119-a500bc606676>:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_full.fillna(method='bfill', axis=0, inplace=True)  # Backward fill


In [ ]:
test_full.head()

,PatientId,AppointmentID,ScheduledDay,AppointmentDay,SMS_received,Hipertension,Diabetes,Alcoholism,Handcap,Gender,Age,Scholarship,Neighbourhood_JARDIM CAMBURI,Neighbourhood_MARIA ORTIZ,Neighbourhood_JARDIM DA PENHA,Neighbourhood_RESISTÊNCIA,Neighbourhood_ITARARÉ
0,7.223290e+12,5640433.0,2016-04-29T10:43:14Z,2016-04-29T00:00:00Z,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0
1,8.632300e+14,5616091.0,2016-04-25T13:29:16Z,2016-04-29T00:00:00Z,1.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0
2,4.438590e+14,5637909.0,2016-04-29T07:30:31Z,2016-04-29T00:00:00Z,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0
3,4.179930e+13,5638018.0,2016-04-29T07:37:45Z,2016-04-29T00:00:00Z,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0
4,3.647760e+13,5614045.0,2016-04-25T10:01:13Z,2016-04-29T00:00:00Z,1.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
drop_cols=['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay']
train= train_full.drop(drop_cols,axis=1)
test= test_full.drop(drop_cols,axis=1)

In [ ]:
# train['No-show'].unique()
train['No-show'].value_counts()

,count
No-show,
No,79360
Yes,20130


In [ ]:
train['No-show'] = train['No-show'].map({'Yes': 1, 'No': 0})

In [ ]:
train['No-show'].unique()
train['No-show'].value_counts()

,count
No-show,
0,79360
1,20130


In [ ]:
train.head()

,SMS_received,No-show,Hipertension,Diabetes,Alcoholism,Handcap,Gender,Age,Scholarship,Neighbourhood_JARDIM CAMBURI,Neighbourhood_MARIA ORTIZ,Neighbourhood_JARDIM DA PENHA,Neighbourhood_RESISTÊNCIA,Neighbourhood_ITARARÉ
0,0,0,1.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,1.0,1.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,1.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
test.head()

,SMS_received,Hipertension,Diabetes,Alcoholism,Handcap,Gender,Age,Scholarship,Neighbourhood_JARDIM CAMBURI,Neighbourhood_MARIA ORTIZ,Neighbourhood_JARDIM DA PENHA,Neighbourhood_RESISTÊNCIA,Neighbourhood_ITARARÉ
0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
test.shape

(10854, 13)

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10854 entries, 0 to 10853
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SMS_received                   10854 non-null  float64
 1   Hipertension                   10854 non-null  float64
 2   Diabetes                       10854 non-null  float64
 3   Alcoholism                     10854 non-null  float64
 4   Handcap                        10854 non-null  float64
 5   Gender                         10854 non-null  float64
 6   Age                            10854 non-null  float64
 7   Scholarship                    10854 non-null  float64
 8   Neighbourhood_JARDIM CAMBURI   10854 non-null  float64
 9   Neighbourhood_MARIA ORTIZ      10854 non-null  float64
 10  Neighbourhood_JARDIM DA PENHA  10854 non-null  float64
 11  Neighbourhood_RESISTÊNCIA      10854 non-null  float64
 12  Neighbourhood_ITARARÉ          10854 non-null 

In [ ]:
# check Dataframe structures to concat
print(train.head()); print(test.head())

In [ ]:
all_data=pd.concat([train,test],axis=0,sort=False)

In [ ]:
all_data.fillna(method='ffill', axis=0, inplace=True)  # Forward fill
all_data.fillna(method='bfill', axis=0, inplace=True)  # Backward fill

<ipython-input-131-0470567a2e3f>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_data.fillna(method='ffill', axis=0, inplace=True)  # Forward fill
<ipython-input-131-0470567a2e3f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_data.fillna(method='bfill', axis=0, inplace=True)  # Backward fill


In [ ]:
print(all_data['No-show'].value_counts())


No-show
0.0    90214
1.0    20130
Name: count, dtype: int64


Model Training and Evaluation
* Split the data: Divide the dataset into training and testing sets.
* Train the model: Use the training set to fit the selected model.
* Evaluate the model: Use the testing set to assess the model's performance.
 Metrics like accuracy, precision, recall, F1-score, and AUC-ROC can be used.

In [ ]:
# splitting & scaling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X = all_data.drop('No-show', axis=1)  # x= features
y = all_data['No-show']  # y= target



In [ ]:
y.value_counts()

,count
No-show,
0.0,90214
1.0,20130


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 42)    # test = variable(val)
scaler = StandardScaler()
X_train_S = scaler.fit_transform(X_train)
X_test_S = scaler.transform(X_test)

In [ ]:
y_train.value_counts()

,count
No-show,
0.0,72151
1.0,16124


In [ ]:
y_test.value_counts()

,count
No-show,
0.0,18063
1.0,4006


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score,accuracy_score, classification_report, confusion_matrix

In [ ]:
RF = RandomForestClassifier()
RF.fit(X_train, y_train)

y_pred = RF.predict(X_test)

score = roc_auc_score(y_test, y_pred)
print(f"Accuracy: {score:.3f}")

Accuracy: 0.508


In [ ]:
# Perform cross-validation for each model
from sklearn.model_selection import cross_val_score, GridSearchCV

scores = cross_val_score(RF, X, y, cv=5, n_jobs=-1)
print(f"{name} accuracy scores: {scores}")
print(f"{name} average accuracy: {scores.mean()}")

Neighbourhood_ITARARÉ accuracy scores: [0.80053469 0.77266754 0.71611763 0.73977072 0.80301794]
Neighbourhood_ITARARÉ average accuracy: 0.7664217048162085


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [5, 10, 15]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X, y)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")


Best Parameters: {'max_depth': 5, 'n_estimators': 10}
Best Score: 0.8175705067188673


In [ ]:
# Make predictions on the test set
# Train the model
RF.fit(X_train, y_train)
# make pridction
test_pred = RF.predict(test)

In [ ]:
# Prepare the submission DataFrame
submission = pd.DataFrame({
    'Patient_ID': test_full['PatientId'],
    'No-show': test_pred.flatten()
})

In [ ]:
submission['No-show'].value_counts()

,count
No-show,
0.0,10802
1.0,52


In [ ]:
# Save to CSV
submission.to_csv('submission.csv', index=False)

In [ ]:
# Check the shape of the submission file
print(submission.shape)

(10854, 2)


In [ ]:
from google.colab import files

# Download the file
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
output_df = pd.read_csv('submission.csv')

In [ ]:
# Basic checks
print("Columns:", output_df.columns)
print("Number of rows:", len(output_df))
print("Data types:\n", output_df.dtypes)
print("Missing values:\n", output_df.isnull().sum())
print("Duplicate values:", output_df.duplicated().sum())

Columns: Index(['Patient_ID', 'No-show'], dtype='object')
Number of rows: 10854
Data types:
 Patient_ID    float64
No-show       float64
dtype: object
Missing values:
 Patient_ID    0
No-show       0
dtype: int64
Duplicate values: 4614


In [ ]:
# Specific checks
print("Unique Patient_ID:", output_df['Patient_ID'].nunique())
print("No-show alues:", output_df['No-show'].unique())

Unique Patient_ID: 6222
No-show alues: [0. 1.]


# This model is correct

# We can also apply tensorflow to check the performance

In [ ]:
# training- tenserflow
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


* Import TensorFlow and Prepare Data
* Define the Model
* Train the Model
* Evaluate the Model

In [ ]:
# Define the tf dataset, Before this, define X, y and also train_test_split
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [ ]:
# Model Building Using Both APIs
""" feature_extractor = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu')
], name='feature_extractor')

# Define Functional model components
inputs = tf.keras.Input(shape=(X_train.shape[1],))

# Use the Sequential model as part of the Functional model
x = feature_extractor(inputs)

# Add more layers using Functional API
x = tf.keras.layers.Dense(32, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Create the Functional model
model = tf.keras.Model(inputs=inputs, outputs=outputs)


# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# Print the model summary
model.summary()"""


" feature_extractor = tf.keras.Sequential([\n    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),\n    tf.keras.layers.Dense(64, activation='relu')\n], name='feature_extractor')\n\n# Define Functional model components\ninputs = tf.keras.Input(shape=(X_train.shape[1],))\n\n# Use the Sequential model as part of the Functional model\nx = feature_extractor(inputs)\n\n# Add more layers using Functional API\nx = tf.keras.layers.Dense(32, activation='relu')(x)\noutputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)\n\n# Create the Functional model\nmodel = tf.keras.Model(inputs=inputs, outputs=outputs)\n\n\n# Compile the model\nmodel.compile(optimizer='adam',\n              loss='binary_crossentropy',\n              metrics=['accuracy'])\n# Print the model summary\nmodel.summary()"

Model Building Using sequential API

In [ ]:

# Define the Model by create a simple neural network with TensorFlow’s Keras API.
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])

# Print the model summary
model.summary()



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,009 (11.75 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test,y_test))

Epoch 1/10
2759/2759 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - AUC: 0.5397 - loss: 0.5276 - val_AUC: 0.5842 - val_loss: 0.4693
Epoch 2/10
2759/2759 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - AUC: 0.5671 - loss: 0.4766 - val_AUC: 0.5904 - val_loss: 0.4668
Epoch 3/10
2759/2759 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - AUC: 0.5769 - loss: 0.4722 - val_AUC: 0.5823 - val_loss: 0.4754
Epoch 4/10
2759/2759 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - AUC: 0.5809 - loss: 0.4692 - val_AUC: 0.5927 - val_loss: 0.4665
Epoch 5/10
2759/2759 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - AUC: 0.5929 - loss: 0.4693 - val_AUC: 0.5950 - val_loss: 0.4655
Epoch 6/10
2759/2759 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - AUC: 0.5867 - loss: 0.4661 - val_AUC: 0.5943 - val_loss: 0.4655
Epoch 7/10
2759/2759 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - AUC: 0.5907 - loss: 0.4649 - val_AUC: 0.5907 - val_loss: 0.4661
Epoch 8/10
2759/2759 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - AUC: 0.5930 - loss: 0.4671 - val_AUC: 0.5962 - val_loss: 0.4656
Epoch 9/10
2759/2759 ━━━━━━━━━━━━━━━

In [ ]:
# Evaluate on the validation set
val_loss, val_auc = model.evaluate(X_test,y_test)
print(f'Validation ROC AUC: {val_auc:.2f}')

690/690 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - AUC: 0.5946 - loss: 0.4684
Validation ROC AUC: 0.60


Make Predictions and Prepare Submission

In [ ]:
# Predict on the Test Set
test_pred = model.predict(test)

340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [ ]:

# Prepare the submission DataFrame
submission = pd.DataFrame({
    'Patient_ID': test_full['PatientId'],
    'No-show': test_pred.flatten()
})

In [ ]:
# Save to CSV
submission.to_csv('submission.csv', index=False)

In [ ]:
# Check the shape of the submission file
print(submission.shape)  # Should match the test dataset

(10854, 2)


In [ ]:
submission['No-show'].unique()

array([0.13615927, 0.24322161, 0.11669313, ..., 0.13598645, 0.1290527 ,
       0.16253786], dtype=float32)

In [ ]:
from google.colab import files

# Download the file
files.download('submission.csv')